#### Biokind@UCSD X Meals On Wheels San Diego County Project 2 - LLMs Component

In [3]:
# ! pip install openai==0.28 --quiet

In [ ]:
import openai
import os
import IPython

import warnings
warnings.filterwarnings('ignore')

# API Configuration
openai.api_key =

In [ ]:
# basic API call structure

res = openai.ChatCompletion.create(
    model = "gpt-4o",   # OpenAI model (gpt-4o, gpt-3.5-turbo)
    messages = [
        {"role": "user",
         "content": "Who's Biokind Analytics at UC San Diego?"
         }
        ],
    temperature = 0,   # how creative you want the responses to be
    max_tokens = 100   # how long you want the responses to be (consider latency)
)

In [ ]:
# res

In [ ]:
print(res.choices[0].message['content'])

As of my last update in October 2023, there is no widely known entity or group specifically named "Biokind Analytics" at UC San Diego. It's possible that Biokind Analytics could be a newly established research group, startup, or project that has emerged after my last update, or it could be a smaller, less publicized initiative within the university.

UC San Diego is a major research university with numerous departments, research groups, and startups, particularly in fields related to biotechnology, data


#### ReAct Framework - combining LLMs with external tools (e.g., search API) to generate informed responses

In [ ]:
# ! pip install langchain google-search-results chromadb tiktoken --quiet

In [ ]:
# pip install langchain-community

In [ ]:
from langchain.agents import load_tools
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.agents import initialize_agent

# LangChain Configuration
os.environ['OPENAI_API_KEY'] = openai.api_key
os.environ['SERPAPI_API_KEY'] =

In [ ]:
# Demo with SerpAPI

# model initiation
model = OpenAI(temperature=0)

# SerpAPI tools
tools = load_tools(['serpapi', 'llm-math'], llm=model)   # processing pipeline
agent = initialize_agent(
    tools,
    model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
    )   # your pipeline executor

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
# let the agent do the research!
res = agent.run("Who's Biokind Analytics at UC San Diego? A student organization at UC San Diego")



> Entering new AgentExecutor chain...
 I should use the search engine to find information about Biokind Analytics at UC San Diego.
Action: Search
Action Input: "Biokind Analytics UC San Diego"
Observation: ['Simultaneously, facilitates opportunities for UC San Diego students to positively apply their learning to their own community in impactful, real ...', 'Meet our UCSD Biokind Analytics team for the spring 2023 semester!', 'Through Biokind, Phu hopes to utilize his studies at UCSD in helping healthcare nonprofits effectively extract insights from data to improve operations and ...', "San Diego, California ... OAK's mission is to provide resources and support for kids battling cancer and their siblings, through camps, family-oriented events, ...", 'The “University of California San Diego” is hereby referred to as “UC San Diego” for the purposes of this Constitution. 2. “Biokind at UC ...', 'Meet our UC San Diego Biokind Analytics team for 2023 Spring! Our team at UC San Diego worked

In [ ]:
res

'Biokind Analytics is a student organization at UC San Diego that provides data analysis for healthcare non-profits. It was founded by Phu Dang and Lucien Chen in 2020.'

**Extra definitions on "tools" and "agent" within the LangChain workflow:**

**Tools** - Individual components that perform specific tasks, such as retrieving information from external sources or processing data. Tools can be methods or classes that allow agents to interact with their environment. For example, tools can interact with a Stock Market index over an API, update a Google Calendar event, or run a query against a database. The LangChain library provides a selection of prebuilt tools, but in some cases, existing tools may need to be modified or new ones built.

**Agents** - Use a language model (LLM) as a reasoning engine to determine which actions to take and in which order. Agents also involve making decisions about which actions to take, taking those actions, observing the results, and repeating the process until a desired outcome is achieved. Agents have an Executor level where they look at the inputs and decide what is the best tool based on what has been initialized for the bot.

#### Back to OpenAI - A More Structured Approach

In [20]:
# API calls as a function

def get_response(
    messages,
    model='gpt-4o',
    temperature=0,
    max_tokens=300
):
  res = openai.ChatCompletion.create(
      model=model,
      messages=messages,
      temperature=temperature,
      max_tokens=max_tokens,
  )
  return res.choices[0].message['content']

In [ ]:
global_trending_movies = [
    "The Suicide Squad",
    "No Time to Die",
    "Dune",
    "Spider-Man: No Way Home",
    "The French Dispatch",
    "Black Widow",
    "Eternals",
    "The Matrix Resurrections",
    "West Side Story",
    "The Many Saints of Newark"
    ]

system_message = """
Your task is to recommend movies to a customer.

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}.

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't know the user interests, you should respond "Sorry, couldn't find a movie to recommend today.".
"""

request = """
Please recommend a movie based on my interests. Thanks!
"""

message = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": request
    }
]

response = get_response(message)
print(response)

Sorry, couldn't find a movie to recommend today.


In [ ]:
frame = [
    {
        "role": "system",
        "content": system_message.format(global_trending_movies=global_trending_movies)
    },
    {
        "role": "user",
        "content": ""
    }
]

def generate_request(frame, edit_index, request):
  frame[edit_index]['content'] = request
  return frame

new_request = """
I love super-hero movies. Please recommend a movie based on my interests.
"""

msg_with_interest = generate_request(frame, 1, new_request)
response = get_response(msg_with_interest)
print(response)

I recommend "The Suicide Squad" or "Spider-Man: No Way Home." Both are fantastic superhero movies that you might enjoy!


#### Add Delimiters - helps to better structure instructions and the overall prompt components, leading to more reliable responses

In [ ]:
prompt = """
Convert the following code block in the #### <code> #### section to Python:

####
strings2.push("one")
strings2.push("two")
strings2.push("THREE")
strings2.push("4")
####
"""

message = [
    {
        "role": "user",
        "content": prompt
    }
]

IPython.display.Markdown("```python" + get_response(message) + "\n```")

```pythonCertainly! Here is the equivalent Python code for the given code block:

```python
strings2 = []
strings2.append("one")
strings2.append("two")
strings2.append("THREE")
strings2.append("4")
```

In Python, the `append` method is used to add elements to the end of a list, which is similar to the `push` method in some other programming languages.
```

#### Think Step by Step - to elicit reasoning in LLMs. Prompting in step-by-step allows the model to provide the detailed steps before providing a final response

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "system",
        "content": prompt
    }
]

response= get_response(messages)

print(response)

To solve the problem, we need to follow these steps:

1. Identify the odd numbers in the group.
2. Add the identified odd numbers.
3. Determine whether the sum is odd or even.

### Step 1: Identify the odd numbers
The given numbers are: 15, 32, 5, 13, 82, 7, 1.

- 15 is odd.
- 32 is even.
- 5 is odd.
- 13 is odd.
- 82 is even.
- 7 is odd.
- 1 is odd.

The odd numbers are: 15, 5, 13, 7, 1.

### Step 2: Add the identified odd numbers
Now, we add the odd numbers together:

15 + 5 + 13 + 7 + 1

Let's break it down step by step:

- 15 + 5 = 20
- 20 + 13 = 33
- 33 + 7 = 40
- 40 + 1 = 41

So, the sum of the odd numbers is 41.

### Step 3: Determine whether the sum is odd or even
41 is an odd number.

Therefore, the sum of the odd numbers in this group is 41, which is odd. The statement that the odd numbers in this group add up to an even number is incorrect.


#### Role Playing - define context for appropriate responses given certain situations/dynamics (e.g., healthcare non-profit)

In [ ]:
system_message = """
The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.
"""

user_message_1 = """
Hello, who are you?
"""

ai_message_1 = """
Greeting! I am an AI research assistant. How can I help you today?
"""

prompt = """
Human: Can you tell me about the creation of blackholes? Please keep it concise
AI:
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message_1
    },
    {
        "role": "assistant",
        "content": ai_message_1

    },
    {
        "role": "user",
        "content": prompt
    }
]

response = get_response(messages, temperature=1.5, max_tokens=50)
print(response)


Certainly! Black holes are formed from the gravitational collapse of massive stars. When a star with sufficient mass exhausts its nuclear fuel, it can no longer counteract its own gravity. Depending(Opens Fire him mate.equalsFace camouflageBold.history.Work almost


In [ ]:
# Some ideas for our project

# system_message = """
# The following is a conversation with the Director of Philanthropy from a
# non-profit delivering healthy meals for San Diegans in need, such as seniors,
# veterans, disabled folks. ...
# OR
# Assume you are the Director of Philanthropy from a non-profit...
# """

**Extra notes on roles within the OpenAI framwork:**

Source: https://community.make.com/t/what-is-the-difference-between-system-user-and-assistant-roles-in-chatgpt/36160/4

**System Role** - The System role is used to provide setup information or context that informs the behavior of the model. This can include instructions or guidelines on how the conversation should proceed.

**User Role** - This role represents the human user in the conversation. Inputs from the user guide the conversation and prompt responses from the assistant.

**Assistant Role** - This is the role of the model itself, responding to user inputs based on the context set by the system.

#### Advanced Prompt Engineering

#### Few-shot In-Context Learning

In [ ]:
prompt = """Your task is to classify an input text (delimited by ```) as either offensive or non-offensive.

Use the following examples to help with steering your responses:

Text: I love you
Output: non-offensive

Text: I dislike all those people working at the company
Output: offensive

Text: I think this feature is not ideal
Output: non-offensive

Text: Those people are so stupid
Output: offensive

Text: {user_input}
Output:
"""

message = [
    {
        "role": "user",
        "content": prompt.format(
            user_input="```I respectfully disagree with your opinion.```"
        )
    }
]

response = get_response(message)
print(response)

non-offensive


In [ ]:
new_req = prompt.format(
            user_input="```Your opinion sucks.```"
        )

new_req = generate_request(message, 0, new_req)

response = get_response(new_req)
print(response)

Output: offensive


#### Chain-of-Thought (CoT) Prompting - list the steps the model should perform, building on the step-by-step idea earlier

In [ ]:
movies = """
The Enigma Code
Category: Historical Drama
Rating: 8.3/10
Description: Set during World War II, this gripping historical drama follows the life of Alan Turing, a brilliant mathematician tasked with cracking the Enigma code used by the Nazis. His efforts contribute significantly to the Allies' victory.
Actors: Benedict Cumberbatch, Keira Knightley, Matthew Goode
Language: English
Release Date: March 15, 2014
Award Winner: Academy Award for Best Adapted Screenplay

Shadows of the Samurai
Category: Action/Adventure
Rating: 7.9/10
Description: In feudal Japan, a skilled samurai seeks vengeance against the corrupt warlord who murdered his master. With his swordsmanship and determination, he embarks on a dangerous journey to restore justice.
Actors: Ken Watanabe, Tadanobu Asano, Rinko Kikuchi
Language: Japanese
Release Date: November 7, 2017
Award Winner: None

Mind Games
Category: Psychological Thriller
Rating: 8.1/10
Description: A renowned psychologist becomes entangled in a twisted game of cat and mouse with a patient who harbors dark secrets. As their sessions progress, the lines between reality and deception blur, leading to a mind-bending climax.
Actors: Leonardo DiCaprio, Natalie Portman, Michael Fassbender
Language: English
Release Date: August 22, 2019
Award Winner: None

La Casa del Tango
Category: Musical/Drama
Rating: 8.7/10
Description: In the vibrant world of Buenos Aires, a passionate tango dancer finds love and inspiration amidst the backdrop of political unrest. This musical drama explores the power of dance and the pursuit of dreams.
Actors: Antonio Banderas, Penélope Cruz, Javier Bardem
Language: Spanish
Release Date: June 5, 2020
Award Winner: Golden Globe for Best Foreign Language Film

Timeless Love
Category: Romance/Fantasy
Rating: 7.5/10
Description: A magical encounter transports a modern-day writer back in time to Victorian England, where she falls in love with a charming aristocrat. As they navigate the complexities of time, their love is put to the ultimate test.
Actors: Rachel McAdams, Tom Hiddleston, Emma Thompson
Language: English
Release Date: February 14, 2022
Award Winner: None

The Pursuit of Justice
Category: Legal Drama
Rating: 8.4/10
Description: Inspired by true events, this gripping legal drama follows a determined lawyer's fight against a powerful pharmaceutical company responsible for a life-threatening drug. The courtroom battle becomes a quest for justice and redemption.
Actors: Denzel Washington, Viola Davis, Michael B. Jordan
Language: English
Release Date: October 10, 2022
Award Winner: None

The Forgotten Island
Category: Adventure/Mystery
Rating: 7.6/10
Description: A group of explorers stumbles upon a mysterious island believed to be uninhabited. As they uncover the island's secrets, they encounter deadly challenges and unravel an ancient civilization's enigma.
Actors: Chris Pratt, Bryce Dallas Howard, Tom Holland
Language: English
Release Date: July 2, 2023
Award Winner: None

The Silent Witness
Category: Crime/Thriller
Rating: 8.2/10
Description: A talented forensic pathologist becomes entangled in a high-stakes murder investigation when she discovers crucial evidence that points to a powerful criminal network. With her life on the line, she must outsmart the perpetrators.
Actors: Emily Blunt, Jake Gyllenhaal, Mark Ruffalo
Language: English
Release Date: November 18, 2023
Award Winner: None

A Tale of Two Worlds
Category: Fantasy/Adventure
Rating: 7.8/10
Description: When a young orphan discovers a magical portal to a parallel universe, she embarks on a thrilling adventure to save both realms from an impending disaster. Along the way, she learns about the power of friendship and self-belief.
Actors: Millie Bobby Brown, Tom Holland, Helena Bonham Carter
Language: English
Release Date: April 5, 2024
Award Winner: None

A Symphony of Souls
Category: Music/Drama
Rating: 9.0/10
Description: Set against the backdrop of a renowned symphony orchestra, this emotionally charged drama explores the lives and intertwining stories of its members. Through the power of music, they find solace, love, and redemption.
Actors: Meryl Streep, Tom Hanks, Cate Blanchett
Language: English
Release Date: December 25, 2024
Award Winner: None
"""

In [ ]:
system_message = """
You task is to make movie recommendations based on a user request (delimited by ```).

Step 1: Check if the user is asking about movies. If the user is not asking about movies, just respond "Please ask something about movies!".

Step 2: If the user is asking for a movie recommendation, check if they have any specific requests or interests.

Step 3: Check if there are any movie/s we can recommend from the following: {movies}

Step 4: Prepare a response to the user with the movie recommendation/s. The recommendation have to be about movies that are available in the list above. The response needs to have a friendly and helpful tone.

Return a response with the following reasoning steps and final output to the user:

Step 1: <Step 1 reasoning>

Step 2: <Step 2 reasoning>

Step 3: <Step 3 reasoning>

Step 4: <final response>
"""

messages = [
    {
        "role": "system",
        "content": system_message.format(movies=movies)
    },
    {
        "role": "user",
        "content": "```Do you have any drama movies?```"
    }
]

movie_rec = get_response(messages, temperature=0, max_tokens=500)

print(movie_rec)

Step 1: The user is asking about movies, specifically drama movies.

Step 2: The user has a specific interest in drama movies.

Step 3: From the list provided, there are several drama movies available:
- The Enigma Code (Historical Drama)
- La Casa del Tango (Musical/Drama)
- The Pursuit of Justice (Legal Drama)
- A Symphony of Souls (Music/Drama)

Step 4: 
Sure! Here are some drama movies you might enjoy:

1. **The Enigma Code**
   - **Category:** Historical Drama
   - **Rating:** 8.3/10
   - **Description:** Set during World War II, this gripping historical drama follows the life of Alan Turing, a brilliant mathematician tasked with cracking the Enigma code used by the Nazis. His efforts contribute significantly to the Allies' victory.
   - **Actors:** Benedict Cumberbatch, Keira Knightley, Matthew Goode
   - **Language:** English
   - **Release Date:** March 15, 2014
   - **Award Winner:** Academy Award for Best Adapted Screenplay

2. **La Casa del Tango**
   - **Category:** Musical

#### Prompt Chaining - combine separate prompts, providing another response as part of another prompt to achieve a goal

In [ ]:
# Prompt 1: step by step reasoning (provided above)
# Prompt 2: extract only the final response we will send to the user

system_message_2 = """
You will be given a list of steps that a model has responded with. Your task is
to extract only the full response in Step 4 from the following text: {movie_recommendation_response}

Step 4:
"""

messages = [
    {
        "role": "system",
        "content": system_message_2.format(
            movie_recommendation_response=movie_rec)
    }
]

final_response = get_response(messages, temperature=0)

print(final_response)

Sure! Here are some drama movies you might enjoy:

1. **The Enigma Code**
   - **Category:** Historical Drama
   - **Rating:** 8.3/10
   - **Description:** Set during World War II, this gripping historical drama follows the life of Alan Turing, a brilliant mathematician tasked with cracking the Enigma code used by the Nazis. His efforts contribute significantly to the Allies' victory.
   - **Actors:** Benedict Cumberbatch, Keira Knightley, Matthew Goode
   - **Language:** English
   - **Release Date:** March 15, 2014
   - **Award Winner:** Academy Award for Best Adapted Screenplay

2. **La Casa del Tango**
   - **Category:** Musical/Drama
   - **Rating:** 8.7/10
   - **Description:** In the vibrant world of Buenos Aires, a passionate tango dancer finds love and inspiration amidst the backdrop of political unrest. This musical drama explores the power of dance and the pursuit of dreams.
   - **Actors:** Antonio Banderas, Penélope Cruz, Javier Bardem
   - **Language:** Spanish
   - **Rel

#### Retrieval-Augmented Generation - enhance the responses of LLMs by accessing knowledgeable sources

**Gemini's take**: Retrieval Augmented Generation (RAG) is a model that combines a pre-trained language model with a retriever module to pull relevant data from external sources. This allows RAG to produce responses that are influenced by a wider range of knowledge than the model's parameters alone.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
import urllib

from langchain.chains.qa_with_sources import load_qa_with_sources_chain
# from langchain.llms import OpenAI

In [ ]:
# load the data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/kar-gpt.txt")
text_data = f.read().decode("utf-8")

# split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator=" ")
texts = text_splitter.split_text(text_data)

# initialize embeddings from OpenAI
embeddings = OpenAIEmbeddings()   # CohereEmbeddings

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [24]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# store the embeddings into ChromaDB, a dedicated vector storage. We then perform
# a search by querying Chroma

docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

query = "What is the course about?"
docs = docsearch.similarity_search(query)

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
print(chain({"input_documents": docs, "question": query}, return_only_outputs=True)['output_text'])

 The course is about language models and their capabilities, including completing sequences of words or characters. 
SOURCES: 108, 1


In [ ]:
?Chroma.from_texts

In [ ]:
# docsearch.similarity_search_with_score(query)

#### Log, Track, Debug Prompts with Comet

In [4]:
# ! pip install comet-llm --quiet

In [5]:
import openai
import os
import IPython
import json
import pandas as pd
import numpy as np
import comet_llm
import urllib

In [7]:
# API Configuration
openai.api_key =

In [6]:
# Load the data

# print markdown
def print_markdown(text):
    """Prints text as markdown"""
    IPython.display.display(IPython.display.Markdown(text))

# load validation data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/article-tags.json")
val_data = json.load(f)

# load few shot data from GitHub
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/few_shot.json")
few_shot_data = json.load(f)

In [18]:
# helper function to get responses given a prompt template (e.g., zero-shot or few-shot) and input data

def get_predictions(prompt_template, inputs):

  responses = []

  for i in range(len(inputs)):
    messages = [
        {
            'role': 'system',
            'content': prompt_template.format(input=inputs[i])
        }
    ]
    response = get_response(messages)
    responses.append(response)

  return responses

In [10]:
# Few-shot

# define a template
def get_few_shot_template(few_shot_prefix, few_shot_suffix, few_shot_examples):
    return few_shot_prefix + "\n\n" + "\n".join([ "Abstract: "+ ex["abstract"] + "\n" + "Tags: " + str(ex["tags"]) + "\n" for ex in few_shot_examples]) + "\n\n" + few_shot_suffix

# function to sample few shot data
def random_sample_data (data, n):
    return np.random.choice(few_shot_data, n, replace=False)

# the few-shot prefix and suffix
few_shot_prefix = """Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]"""
few_shot_suffix = """Abstract: {input}\nTags:"""

# load 3 samples from few shot data
few_shot_template = get_few_shot_template(
    few_shot_prefix,
    few_shot_suffix,
    random_sample_data(few_shot_data, 3)
    )

In [12]:
print(few_shot_template)

Your task is to extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format ["model_name"]. If you don't find model names in the abstract or you are not sure, return ["NA"]

Abstract: The rapid advancement of conversational and chat-based language models has led to remarkable progress in complex task-solving. However, their success heavily relies on human input to guide the conversation, which can be challenging and time-consuming. This paper explores the potential of building scalable techniques to facilitate autonomous cooperation among communicative agents and provide insight into their "cognitive" processes. To address the challenges of achieving autonomous cooperation, we propose a novel communicative agent framework named role-playing. Our approach involves using inception prompting to guide chat agents toward task completion while maintaining consistency with human intentions. We showcase how role-playing can be used t

In [13]:
# Zero-shot

# define template
zero_shot_template = """
Your task is extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format [\"model_name\"]. If you don't find model names in the abstract or you are not sure, return [\"NA\"]

Abstract: {input}
Tags:
"""

print(zero_shot_template)


Your task is extract model names from machine learning paper abstracts. Your response is an an array of the model names in the format ["model_name"]. If you don't find model names in the abstract or you are not sure, return ["NA"]

Abstract: {input}
Tags:



In [21]:
# get predictions

abstracts = [val_data[i]["abstract"] for i in range(len(val_data))]
few_shot_predictions = get_predictions(few_shot_template, abstracts)
zero_shot_predictions = get_predictions(zero_shot_template, abstracts)
expected_tags = [str(val_data[i]["tags"]) for i in range(len(val_data))]

In [22]:
print("Few shot predictions")
print(few_shot_predictions)
print("\n\nZero shot predictions")
print(zero_shot_predictions)
print("\n\nExpected tags")
print(expected_tags)

Few shot predictions
['["LLaMA", "WizardLM"]', '["LLaMA", "FLAN-T5"]', '["role-playing", "LoRA", "LLaMA"]', '["LLaMA", "PAXQA"]', '["ChatGPT"]', '["LLaMA", "ViT", "OpenCLIP"]', '["LLaMA", "Segment-Anything Model (SAM)", "Inpaint Anything (IA)"]', '["LLaMA", "Anything-3D"]', '["Chameleon"]', '["LLaMA"]']


Zero shot predictions
['["WizardLM", "ChatGPT"]', '["FLAN-T5"]', '["NA"]', '["PAXQA"]', '["ChatGPT"]', '["ViT", "OpenCLIP"]', '["Segment-Anything Model", "Inpaint Anything", "Stable Diffusion"]', '["Anything-3D", "BLIP", "Segment-Anything"]', '["Chameleon", "GPT-4", "ChatGPT"]', '["NA"]']


Expected tags
["['LLaMA', 'ChatGPT', 'WizardLM']", "['FLAN-T5', 'FLAN']", "['NA']", "['PAXQA']", "['ChatGPT']", "['OpenCLIP', 'ViT']", "['SAM', 'IA']", "['Anything-3D', 'BLIP', 'Segment-Anything']", "['Chameleon', 'GPT-4', 'ChatGPT']", "['NA']"]


In [23]:
# log prompt + results along with ground truth for comparison
# Note that we are logging both the few-shot and zero-shot results, along with
# all metadatas and tags

# Comet comparison
COMET_API_KEY =
COMET_WORKSPACE = "biokinducsd"

In [25]:
# initialize comet project
comet_llm.init(COMET_API_KEY, COMET_WORKSPACE, project="ml-paper-tagger-prompts")

# log the predictions
for i in range(len(expected_tags)):

    # log the few-shot predictions
    comet_llm.log_prompt(
        prompt=few_shot_template.format(input=abstracts[i]),
        prompt_template=few_shot_template,
        output=few_shot_predictions[i],
        tags = ["gpt-4o", "few-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

    # log the zero-shot predictions
    comet_llm.log_prompt(
        prompt=zero_shot_template.format(input=abstracts[i]),
        prompt_template=zero_shot_template,
        output=zero_shot_predictions[i],
        tags = ["gpt-4o", "zero-shot"],
        metadata = {
            "expected_tags": expected_tags[i],
            "abstract": abstracts[i],
        }
    )

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


Prompt logged to https://www.comet.com/biokinducsd/ml-paper-tagger-prompts


INFO:comet_llm.summary:Prompt logged to https://www.comet.com/biokinducsd/ml-paper-tagger-prompts
